## Message Translator (PayloadTransformer from YAML spec)

This notebook demonstrates the Message Translator pattern using a `PayloadTransformer` defined in a YAML guild specification. A Message Translator converts messages from one format to another, enabling agents with different message formats to communicate effectively.

**Input Format**: `CustomerRequest` - A customer service request  
**Output Format**: `InternalTicket` - An internal support ticket format

**YAML Transformation Benefits:**
- **Declarative Configuration**: Guild specification is defined in YAML
- **Variable Assignment**: Uses JSONata variable assignments for better readability
- **Reusable Intermediate Results**: Complex expressions broken into named variables
- **Clean Separation**: Business logic separated from infrastructure code

The YAML specification includes a structured JSONata expression that:
1. Assigns intermediate variables for ticket ID generation
2. Maps priority to severity levels using conditional logic
3. Creates customer information objects
4. Builds the final transformed message

The flow will be:

[ProbeAgent] --`CustomerRequest`-> (default_topic) -> [Translator Agent/TranslatorAgent:wire_message] --`InternalTicket`-> (internal_tickets)

In [ ]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

# Input message format - what customers send
class CustomerRequest(BaseModel):
    customer_name: str
    email: str
    subject: str
    description: str
    priority: str  # "low", "medium", "high"

# Output message format - internal ticket system
class InternalTicket(BaseModel):
    ticket_id: str
    customer_info: dict
    title: str
    details: str
    severity: int  # 1=high, 2=medium, 3=low
    created_at: str
    status: str = "open"

In [ ]:
from rustic_ai.core.guild.builders import GuildBuilder

# Load the guild from YAML specification
# The YAML file contains the JSONata expression for payload transformation
guild = GuildBuilder.from_yaml_file("./005_message_translator_payload.yaml").launch("myorg")

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test message translation.")
    .add_additional_topic("internal_tickets")
    .build()
)

guild._add_local_agent(probe_agent)

In [ ]:
# Send a customer request
customer_request = CustomerRequest(
    customer_name="John Doe",
    email="john.doe@example.com",
    subject="Login Issue",
    description="I can't log into my account. Getting 'invalid credentials' error.",
    priority="high"
)

probe_agent.publish_with_guild_route(
    payload=customer_request,
    topic=guild_default_topic,
)

In [ ]:
# Check the translated message
probe_agent.print_message_history()

In [ ]:
last_message = probe_agent.get_messages()[-1]
print(f"Format: {last_message.format}")
print(f"Payload: {last_message.payload}")

In [ ]:
# Send another customer request with different priority
customer_request_2 = CustomerRequest(
    customer_name="Jane Smith",
    email="jane.smith@company.com",
    subject="Feature Request",
    description="Would like to request a new dashboard feature for analytics.",
    priority="low"
)

probe_agent.publish_with_guild_route(
    payload=customer_request_2,
    topic=guild_default_topic,
)

In [ ]:
# Check all messages to see both translations
probe_agent.print_message_history()

In [ ]:
last_message = probe_agent.get_messages()[-1]
print(f"Format: {last_message.format}")
print(f"Payload: {last_message.payload}")